## AWS credentials for colab notebook

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! pip install awscli

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.9 MB 4.3 MB/s 
     |████████████████████████████████| 8.8 MB 45.6 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |████████████████████████████████| 548 kB 59.7 MB/s 
     |████████████████████████████████| 138 kB 59.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: rsa
    Found existing installation: rsa 4.8
    Uninstalling rsa-4.8:
      Successfully uninstalled rsa-4.8
  Attempting uninstall: docutils
    Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

In [6]:
text = '''
[default]
aws_access_key_id = 
aws_secret_access_key = 
aws_session_token=
region = us-east-1
'''
path = "/content/drive/My Drive/config/awscli.ini"
with open(path, 'w') as f:
   f.write(text)

In [7]:
import os
! export AWS_SHARED_CREDENTIALS_FILE=/content/drive/My\ Drive/config/awscli.ini
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path

In [13]:
! aws s3 cp s3://lsc-projct ./data/ --recursive

download: s3://lsc-projct/3.csv to data/3.csv                     
download: s3://lsc-projct/4.csv to data/4.csv                 
download: s3://lsc-projct/2.csv to data/2.csv                 
download: s3://lsc-projct/5.csv to data/5.csv                 


## Import libraries

In [9]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.1.2 spark-nlp

     |████████████████████████████████| 212.4 MB 61 kB/s 
     |████████████████████████████████| 145 kB 32.3 MB/s 
     |████████████████████████████████| 198 kB 43.0 MB/s 


In [10]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [11]:
spark = sparknlp.start()

In [14]:
data = spark.read.option("header", True).option("multiline", True).option("escape","\"") \
        .csv('./data/*.csv')

df = data.na.drop(subset=['content'])

## Emotion

In [15]:
MODEL_NAME='classifierdl_use_emotion'

documentAssembler = DocumentAssembler()\
    .setInputCol("content")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = ClassifierDLModel.pretrained(name=MODEL_NAME)\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_emotion download started this may take some time.
Approximate size to download 21.3 MB
[OK!]


In [16]:
empty_df = spark.createDataFrame([['']]).toDF("text")
pipelineModel = nlpPipeline.fit(empty_df)
result = pipelineModel.transform(df)

In [17]:
res = result.select(F.explode(F.arrays_zip('document.result', 'sentiment.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("sentiment"))


In [18]:
res.groupby('sentiment').count().show()

+---------+-----+
|sentiment|count|
+---------+-----+
|      joy| 4429|
|     fear| 2876|
| surprise|  733|
|  sadness|  379|
+---------+-----+



In [19]:
res.select('document').filter(F.col('sentiment')=='joy').show(10, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|document                                                                                                                                                                                                                                                                                   |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Don't forget! GiveBIG is May 3-4. Make your donation to #habitatskc to provide #affordablehousing solutions for King County families.

Early 

In [20]:
res.select('document').filter(F.col('sentiment')=='fear').show(10, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|document                                                                                                                                                                                                                                                                               |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|“The timing is right for this place to rise.” Petition for 
@NJGov
 to reinstate the monarchy with 
@IAMQUEENLATIFAH
 #Newark  #NJ #affordablehousing #QU

In [21]:
res.select('document').filter(F.col('sentiment')=='surprise').show(10, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|document                                                                                                                                                                                                                                                                                                                                     |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
res.select('document').filter(F.col('sentiment')=='sadness').show(10, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|document                                                                                                                                                                                                                                                                                                                                              |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Sarcasm

In [ ]:
MODEL_NAME='classifierdl_use_sarcasm'

documentAssembler = DocumentAssembler()\
    .setInputCol("content")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = ClassifierDLModel.pretrained(name=MODEL_NAME)\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_sarcasm download started this may take some time.
Approximate size to download 21.3 MB
[OK!]


In [ ]:
empty_df = spark.createDataFrame([['']]).toDF("content")
pipelineModel = nlpPipeline.fit(empty_df)
result = pipelineModel.transform(df)

In [ ]:
res = result.select(F.explode(F.arrays_zip('document.result', 'sentiment.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("sentiment"))


In [ ]:
res.groupby('sentiment').count().show()

+---------+-----+
|sentiment|count|
+---------+-----+
|   normal| 8265|
|  sarcasm|  152|
+---------+-----+



In [ ]:
res.select('document').filter(F.col('sentiment')=='sarcasm').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|document                                                                                                                                                                                                                                                                               |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|43% of students at four-year universities experienced housing insecurity in 2020 and 14% of students experienced #homelessness in the last year.

For som